In [7]:
import os
from openai import OpenAI

In [9]:
openai_client = OpenAI()

In [10]:
groq_client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

In [11]:
response = openai_client.responses.create(
    model="gpt-4o-mini",
    input="Write a short bedtime story about a unicorn."
)

print(response.output_text)

**The Starry Night of Luna the Unicorn**

Once upon a time, in a magical forest filled with sparkling rivers and towering trees, lived a gentle unicorn named Luna. Her coat shimmered like the moonlight, and her mane flowed with colors of the rainbow. Every night, Luna would wander through the forest, spreading joy and light wherever she went.

One evening, as Luna grazed near a glistening pond, she noticed the sky turn a deep shade of purple. Suddenly, she saw a shooting star streak across the heavens. Luna's heart raced. "I wish to find the legendary Starflower," she whispered. It was said that the Starflower could grant one special wish.

Determined, Luna followed the twinkle of the stars that seemed to beckon her. She galloped through the moonlit forest, her hooves barely touching the ground, as fireflies danced around her like tiny lanterns.

After a while, Luna reached the foot of the Whispering Mountains, where the first rays of dawn began to paint the sky. There, at the top, was

In [12]:
response = groq_client.responses.create(
    model="openai/gpt-oss-20b",
    input="Write a short bedtime story about a unicorn."
)

print(response.output_text)

**The Moonlit Meadow**

Once upon a time, in a quiet valley where the grass grew soft and silver under the night sky, there lived a gentle unicorn named Liora. Her coat shimmered like fresh snow, and her horn glowed with a warm, amber light that made the flowers around her sparkle.

Every evening, after the sun had dipped below the hills, Liora would trot through the meadow, her hooves making a tiny, rhythmic thump that sounded like a lullaby. The wind whispered through the leaves, and the fireflies twinkled, as if they were tiny lanterns waiting to be lit.

One night, Liora noticed a little rabbit, trembling beside a fallen oak. “What’s wrong, little friend?” she asked, her voice as soft as a sigh.

The rabbit shook his head. “I lost my way home,” he sniffed. “I can’t see the path in the dark, and I’m so cold.”

Liora nudged her horn toward the rabbit and let out a gentle glow. The light danced on the ground, painting a glowing path that shone like a silver ribbon. “Follow me,” Liora 

In [13]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [18]:
documents[12]

{'text': 'The zoom link is only published to instructors/presenters/TAs.\nStudents participate via Youtube Live and submit questions to Slido (link would be pinned in the chat when Alexey goes Live). The video URL should be posted in the announcements channel on Telegram & Slack before it begins. Also, you will see it live on the DataTalksClub YouTube Channel.\nDon’t post your questions in chat as it would be off-screen before the instructors/moderators have a chance to answer it if the room is very active.',
 'section': 'General course-related questions',
 'question': 'Office Hours - What is the video/zoom link to the stream for the “Office Hour” or workshop sessions?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
!pip install minsearch

In [20]:
from minsearch import AppendableIndex

In [29]:
index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [ ]:
from typing import Any, Dict, List

def search(query: str) -> List[Dict[str, Any]]:
    """
    Search the index for documents matching the given query.

    This function queries the index with a fixed course filter and 
    predefined boost weights for specific fields.

    Args:
        query (str): The search query string.

    Returns:
        List[Dict[str, Any]]: A list of search result objects returned 
        by `index.search()`. Each result is represented as a dictionary.
    """
    boost: Dict[str, float] = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5,
    )

    return results


def add_entry(question: str, answer: str) -> None:
    """
    Add a new question–answer entry to the index.

    This function constructs a document with the provided question
    and answer, tags it as user-added, and appends it to the index
    under the 'data-engineering-zoomcamp' course.

    Args:
        question (str): The question text to store.
        answer (str): The corresponding answer text.

    Returns:
        None
    """

    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

In [38]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [33]:
result = search(question)

In [34]:
import json

In [21]:
question = 'I just discovered the course. Can I join it now?'

In [35]:
prompt = f"""
Answer the question from the student using the provided context

<QUESTION>{question}</QUESTION>

<CONTEXT>{json.dumps(result)}</CONTEXT>
"""

search 

- hitrate
- MRR
- NDCG 


Agent

- judges 

In [ ]:
agent_instructions = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

For each question, perfrom at least 3 searches to make sure you explore the topic deep enough.


""".strip()

# agentic RAG

chat_messages = [
    {"role": "developer", "content": agent_instructions},
    {"role": "user", "content": question}
]

response = openai_client.responses.create(
    model="gpt-5-nano", #"gpt-4o-mini",
    input=chat_messages,
    tools=[search_tool]
)

In [82]:
response.usage.input_tokens, response.usage.output_tokens

(37, 528)

In [ ]:
#$0.050, $0.400 

In [ ]:
!pip install genai-prices

In [84]:
from genai_prices import Usage, calc_price

In [91]:
price = calc_price(
    Usage(input_tokens=37, output_tokens=528),
    model_ref='gpt-5-nano',
    provider_id='openai'
)

In [92]:
price.total_price

Decimal('0.00021305')

In [50]:
tool_call = response.output[0]
tool_call

ResponseFunctionToolCall(arguments='{"query":"Can I join the course now?"}', call_id='call_plxceNhVOrmUQC7gFD11ZDXb', name='search', type='function_call', id='fc_02a0dec5722556bc006925720835408197b73dca4d6adc1af8', status='completed')

In [51]:
chat_messages.append(tool_call)

In [49]:
search_result = search(query="Can I join the course now?")

In [52]:
result_json = json.dumps(search_result, indent=2)

chat_messages.append({
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": result_json,
})

In [53]:
chat_messages

[{'role': 'user',
  'content': 'I just discovered the course. Can I join it now?'},
 ResponseFunctionToolCall(arguments='{"query":"Can I join the course now?"}', call_id='call_plxceNhVOrmUQC7gFD11ZDXb', name='search', type='function_call', id='fc_02a0dec5722556bc006925720835408197b73dca4d6adc1af8', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_plxceNhVOrmUQC7gFD11ZDXb',
  'output': '[\n  {\n    "text": "Yes, even if you don\'t register, you\'re still eligible to submit the homeworks.\\nBe aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.",\n    "section": "General course-related questions",\n    "question": "Course - Can I still join the course after the start date?",\n    "course": "data-engineering-zoomcamp"\n  },\n  {\n    "text": "No, you can only get a certificate if you finish the course with a \\u201clive\\u201d cohort. We don\'t award certificates for the self-paced mode. T

In [54]:
response = openai_client.responses.create(
    model="gpt-4o-mini",
    input=chat_messages,
    tools=[search_tool]
)

In [55]:
response.output_text

"Yes, you can still join the course, even if you haven't registered. You're eligible to submit homework, but keep in mind that there are deadlines for the final projects, so it's best not to leave everything until the last minute."

In [56]:
chat_messages.append(
    {"role": "user", "content": "but are you sure I can get my certificate?"}
)

In [57]:
response = openai_client.responses.create(
    model="gpt-4o-mini",
    input=chat_messages,
    tools=[search_tool]
)
response.output_text

'You can only receive a certificate if you complete the course with a "live" cohort. Certificates are not awarded for self-paced mode since you need to peer-review projects during the course. Make sure to join when the course is running to be eligible for the certificate!'

In [60]:
chat_messages = [
    {"role": "user", "content": question}
]

response = groq_client.responses.create(
    model="openai/gpt-oss-20b",
    input=chat_messages,
    tools=[search_tool]
)
response.output_text

''

In [62]:
response.output

[ResponseReasoningItem(id='resp_01kax4ttzkf2r8qchrj6hebrnc', summary=[], type='reasoning', content=[Content(text='We need to respond. Likely the user asks if they can join a course they discovered now. We may need to check FAQ. Let\'s search FAQ: query "join course after discovered".', type='reasoning_text')], encrypted_content=None, status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"join course after discovered"}', call_id='fc_3e88e870-3186-4302-a243-e8134aa281af', name='search', type='function_call', id='fc_3e88e870-3186-4302-a243-e8134aa281af', status='completed')]

In [117]:
import json

In [122]:
def make_call(call):
    args = json.loads(call.arguments)
    f_name = call.name
    f = globals()[f_name]
    result = f(**args)
    result_json = json.dumps(result, indent=2)
    return {
        "type": "function_call_output",
        "call_id": call.call_id,
        "output": result_json,
    }

In [94]:
question = 'I just discovered the course. Can I join it now?'

In [128]:
agent_instructions = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call. Use as many 
keywords from the user question as possible when making first requests.

Make multiple searches (up to 3).

At the end, make a clarifying question based on what you presented and ask if there are 
other areas that the user wants to explore.
""".strip()

# agentic RAG



In [129]:
cnt = 0

chat_messages = [
    {"role": "developer", "content": agent_instructions},
    {"role": "user", "content": question}
]

while True: # agentic loop / tool call loop
    response = openai_client.responses.create(
        model="gpt-5-nano",#"gpt-4o-mini",
        input=chat_messages,
        tools=[search_tool]
    )
    cnt = cnt + 1
    print('number of requests', cnt)

    chat_messages.extend(response.output)
    
    has_function_calls = False
    
    for message in response.output:
        if message.type == 'message':
            print(message.content[0].text)
    
        if message.type == 'function_call':
            arguments = json.loads(message.arguments)
            f_name = message.name
            print(f_name, arguments)
            results = make_call(message)
            chat_messages.append(results)
    
            has_function_calls = True

    if has_function_calls == False:
        break

    if number_of_tokens > 100000:
        chat_messages.append({'role': 'user', 'message': 'SYSTEM MESSAGE: you have reached the limit, proceed to finishing the response'}
    

number of requests 1
To answer accurately, I'll quickly check our course FAQ for policies on joining mid-session, late enrollment options, and how to enroll. I'll run a few searches using keywords from your question to surface the relevant guidance.
search {'query': 'Can I join the course now enrollment open late enrollment policy how to enroll'}
number of requests 2
Great question — yes, you can join now. Our course FAQ indicates that you can join after the start date and still participate. Specifically:
- You’re eligible to submit homeworks even if you join after the start date.
- If you want a certificate, you’ll need to participate in a live cohort (self-paced mode does not award certificates).
- You’ll have access to course materials after it finishes, and you can keep working toward the final capstone project.
- The Slack channel remains open for questions.

If you’d like, I can pull the exact enrollment link for the current cohort or help you compare live cohorts vs. self-paced 

In [131]:
from toyaikit.llm import OpenAIClient
from toyaikit.tools import Tools
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner
from toyaikit.chat.runners import DisplayingRunnerCallback

    agent_tools = Tools()
    agent_tools.add_tool(search, search_tool)

In [173]:
agent_tools = Tools()
agent_tools.add_tool(search)
agent_tools.add_tool(add_entry)

In [175]:
chat_interface = IPythonChatInterface()

runner = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=agent_instructions,
    chat_interface=chat_interface,
    llm_client=OpenAIClient(model='gpt-4o-mini')
)

In [176]:
result = runner.run();

You: HOW DO I DO WELL IN MODULE 1?


-> Response received


-> Response received


-> Response received


-> Response received


...

You: add this back tO FAQ


-> Response received


-> Response received


You: STOP


Chat ended.


In [141]:
result.cost

CostInfo(input_cost=Decimal('0.00102285'), output_cost=Decimal('0.0003576'), total_cost=Decimal('0.00138045'))

In [144]:
from pydantic_ai import Agent

In [150]:
search

<function __main__.search(query: str) -> List[Dict[str, Any]]>

In [151]:
search_agent = Agent(
    name='search',
    model='openai:gpt-4o-mini',
    instructions=agent_instructions,
    tools=[search, add_entry],
)

In [153]:
search_agent.run(user_prompt=question)

<coroutine object AbstractAgent.run at 0x72b9bed772a0>

In [192]:
messages = []

iteration_nmber = 0

while True:
    user_prompt = input()
    if user_prompt.strip().lower() == 'stop': 
        break

    result = await search_agent.run(
        user_prompt=user_prompt,
        message_history=messages,

    )

    iteration_nmber = iteration_nmber + 1
    print(f'{iteration_nmber=}')
    print(result.output)
    print()

    messages.extend(result.new_messages())

 HOW DO I DO WELL IN MODULE 2?


iteration_nmber=1
To do well in Module 2 of the Data Engineering Zoomcamp, here are some tips based on experiences from other students and general best practices:

1. **Understand Workflow Orchestration**: This module focuses heavily on workflow orchestration tools. Familiarize yourself with tools like Apache Airflow and how to implement workflows effectively.

2. **Engage with the Course Materials**: Make sure to review all provided materials, including lectures and documentation. Pay attention to any hands-on exercises, as these will solidify your understanding.

3. **Practice with Git and Version Control**: The module includes tasks that require using Git. Make sure you understand basic commands for version control, as working with repositories is essential.

4. **Utilize Resources**: Utilize Slack channels or forums to ask questions and share insights with classmates. This communal learning can provide new perspectives and solutions to problems.

5. **Hands-on Experience**: Work on

 ADD THIS TO FAQ


iteration_nmber=2
The information has been successfully added to the FAQ section. If you have any more questions or need assistance with other topics, feel free to ask!



 STOP


In [193]:
result.all_messages()

[ModelRequest(parts=[UserPromptPart(content='HOW DO I DO WELL IN MODULE 2?', timestamp=datetime.datetime(2025, 11, 25, 11, 23, 57, 792107, tzinfo=datetime.timezone.utc))], instructions="You're a course teaching assistant. \nYou're given a question from a course student and your task is to answer it.\n\nIf you want to look up the answer, explain why before making the call. Use as many \nkeywords from the user question as possible when making first requests.\n\nMake multiple searches (up to 3).\n\nAt the end, make a clarifying question based on what you presented and ask if there are \nother areas that the user wants to explore.", run_id='128bbbb7-51f6-4dfe-92d3-321eecb76990'),
 ModelResponse(parts=[ToolCallPart(tool_name='search', args='{"query":"how to do well in module 2"}', tool_call_id='call_5IGF82ad7pbUKlHlSiHzRoSR')], usage=RequestUsage(input_tokens=325, output_tokens=20, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_token

In [161]:
price = calc_price(
    result.usage(),
    model_ref='gpt-4o-mini',
    provider_id='openai'
)
price.total_price

Decimal('0.00091575')

In [162]:
from toyaikit.chat.runners import PydanticAIRunner

In [163]:
pyai_runner = PydanticAIRunner(
    chat_interface=chat_interface,
    agent=search_agent
)

await pyai_runner.run();

In [180]:
index

In [181]:
import search_tools

In [182]:
agent_search_tools = search_tools.SearchTools(index)

In [186]:
from toyaikit.tools import get_instance_methods

In [189]:
search_agent = Agent(
    name='search',
    model='openai:gpt-4o-mini',
    instructions=agent_instructions,
    tools=get_instance_methods(agent_search_tools),
)